In [9]:
!pip install requests

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import zipfile
import requests
import os

# Define the URLs of the .7z files in your repository
train_url = 'https://github.com/sm2hashmi/ML-and-DL-projects/raw/main/datasets/mnist_train.zip'
test_url = 'https://github.com/sm2hashmi/ML-and-DL-projects/raw/main/datasets/mnist_test.zip'

# Define the paths to save the downloaded .zip files
train_zip_path = 'mnist_train.zip'
test_zip_path = 'mnist_test.zip'

# Download the .zip files
def download_file(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Download complete for {save_path}.")
    else:
        print(f"Failed to download file {save_path}. Status code:", response.status_code)

download_file(train_url, train_zip_path)
download_file(test_url, test_zip_path)

# Extract the .zip files
def extract_zip(file_path, extract_path='.'):
    if os.path.isfile(file_path):
        try:
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            print(f"Extraction complete for {file_path}.")
        except zipfile.BadZipFile:
            print(f"Error: The file {file_path} is not a valid zip archive.")
    else:
        print(f"Error: File {file_path} does not exist.")

extract_zip(train_zip_path)
extract_zip(test_zip_path)

# Verify the extracted files
print("Files extracted:", os.listdir('.'))

# Load the datasets
import pandas as pd

train_df = pd.read_csv('mnist_train.csv')
test_df = pd.read_csv('mnist_test.csv')

# Check the first few rows of the dataframes
print(train_df.head())
print(test_df.head())



Download complete for mnist_train.zip.
Download complete for mnist_test.zip.
Extraction complete for mnist_train.zip.
Extraction complete for mnist_test.zip.
Files extracted: ['.config', 'mnist_test.zip', '.ipynb_checkpoints', 'mnist_train.csv', 'mnist_test.csv', 'mnist_train.zip', 'sample_data']
   label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0      5    0    0    0    0    0    0    0    0    0  ...      0      0   
1      0    0    0    0    0    0    0    0    0    0  ...      0      0   
2      4    0    0    0    0    0    0    0    0    0  ...      0      0   
3      1    0    0    0    0    0    0    0    0    0  ...      0      0   
4      9    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3    

In [11]:
# Separate features and labels
X_train = train_df.drop('label', axis=1).values
y_train = train_df['label'].values
X_test = test_df.drop('label', axis=1).values
y_test = test_df['label'].values

# Normalize the pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Reshape the data to fit the model
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((48000, 28, 28, 1),
 (48000, 10),
 (12000, 28, 28, 1),
 (12000, 10),
 (10000, 28, 28, 1),
 (10000, 10))

In [12]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)

test_loss, test_acc

Epoch 1/10
1500/1500 [==============================] - 53s 35ms/step - loss: 0.2807 - accuracy: 0.9133 - val_loss: 0.0852 - val_accuracy: 0.9737
Epoch 2/10
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0922 - accuracy: 0.9746 - val_loss: 0.0499 - val_accuracy: 0.9853
Epoch 3/10
1500/1500 [==============================] - 50s 33ms/step - loss: 0.0618 - accuracy: 0.9824 - val_loss: 0.0501 - val_accuracy: 0.9868
Epoch 4/10
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0532 - accuracy: 0.9850 - val_loss: 0.0360 - val_accuracy: 0.9893
Epoch 5/10
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0426 - accuracy: 0.9877 - val_loss: 0.0358 - val_accuracy: 0.9901
Epoch 6/10
1500/1500 [==============================] - 48s 32ms/step - loss: 0.0343 - accuracy: 0.9901 - val_loss: 0.0382 - val_accuracy: 0.9900
Epoch 7/10
1500/1500 [==============================] - 51s 34ms/step - loss: 0.0312 - accuracy: 0.9906 - val_loss: 0.0420 -

(0.033705372363328934, 0.9912999868392944)

The model's accuracy turns out to be 99.12%, making it very efficient at recognizing handwritten digits. The loss of 3.37% makes it decent enough when dealing with unseen data.